In [7]:
from obspy.clients.fdsn import Client
from obspy import UTCDateTime
import matplotlib
import matplotlib.pyplot as plt
import numpy as np
from obspy.io.quakeml.core import Unpickler
import pandas as pd
from libcomcat.dataframes import get_phase_dataframe, get_magnitude_data_frame
from libcomcat.search import get_event_by_id
import os

client = Client("IRIS")

mseed_savedir = '/data/sdybing/olderdata/mseeds/'
pre_post_sec = 60

ModuleNotFoundError: No module named 'libcomcat'

In [ ]:
print(pd.__version__)

In [ ]:
new_catalog = np.genfromtxt('/home/sdybing/mlaapde/2000-2013_catalog.csv', delimiter = ',', dtype = 'str', skip_header = 1) # M7.5+ 01/01/2000 - 07/31/2013

In [ ]:
new_catalog[:,8]

In [ ]:
# AGENCY.DEPLOYMENT.STATION.--: LAT LON DEPTH BEGTIME ENDTIME

IR_stations = np.genfromtxt('/home/sdybing/mlaapde/FDSN_IRstatcoord.dat', dtype = 'str', delimiter = ' ', usecols = (0,1))

IR_stations_test = IR_stations[29:34]

In [ ]:
# NET STAT LOC COMP: NORIENT EORIENT VERTORIENT: LAT LON DEPTH BEGTIME to ENDTIME

modern_stations = np.genfromtxt('/home/sdybing/mlaapde/ALL_BHZstatcoord.dat', dtype = 'str', usecols = (0,1,2,3,7,8))

bad_format = ['AXBA1', 'AXCC1', 'AXEC2', 'HYSB1']

In [ ]:
IR_stas_reshape = []

for idx in range(len(IR_stations)):
    IR_sta = IR_stations[idx][0].split(':')[0].split('.')[2]
    IR_stas_reshape.append(IR_sta)

IR_stasonly = np.array(IR_stas_reshape)

In [ ]:
# Tohoku

# event_id = 'official20110311054624120_30'

# event_savedir = mseed_savedir + event_id + '/'
# if os.path.isdir(event_savedir):
#     pass
# else: # deletes directory to start over: shutil.rmtree(save_dir)  
#     os.makedirs(event_savedir)

# detail = get_event_by_id(event_id, includesuperseded = True)
# print(detail)

# df = get_phase_dataframe(detail)
# phase_data = df.to_numpy()

In [ ]:
hist_nets = []
hist_stas = []
hist_lats = []
hist_lons = []

In [ ]:
event_ids = new_catalog[:,8]

for event_id in event_ids[0:1]:
    
    print('----------------------------------------------------------------------')
#     event_savedir = mseed_savedir + event_id + '/'
#     if os.path.isdir(event_savedir):
#         pass
#     else: # deletes directory to start over: shutil.rmtree(save_dir)  
#         os.makedirs(event_savedir)

    # Get the phase picks from LibComCat
    detail = get_event_by_id(event_id, includesuperseded = True)
    print(detail)
    df = get_phase_dataframe(detail)
    phase_data = df.to_numpy()
    
    # Figure out the new station identifier and download and save data when possible
    mult_locs = []
    counter = 0

    for idx in range(len(phase_data)):
        print('--------------------------------')
        counter += 1
        print('Phase pick ' + str(counter) + '/' + str(len(phase_data)))
        row = phase_data[idx]
        net = row[0][0:2]
        sta = row[0][3:-6]
        phase = row[3]
        arrtime = UTCDateTime(row[4])

    #     print(str(net) + ', ' + str(sta) + ', ' + str(phase) + ', ' + str(arrtime))

        p = np.where(IR_stasonly == sta)[0]
    #     print(p)

        if len(p) == 0: # if the station associated with the phase pick isn't in the IR list
            IR_sta = sta

        elif len(p) > 1: # if there are multiple entries for the same station in the IR list because of multiple locations
            IR_sta = IR_stations[p[0]][0].split(':')[0].split('.')[2]
            IR_lat = float(IR_stations[p[0]][0].split(':')[1])
            IR_lon = float(IR_stations[p[0]][1])
            mult_locs.append(str(IR_sta) + '_num_locs:' + str(len(p)))

        else: # if len(p) = 1 (or negative numbers but that wouldn't happen)
            IR_sta = IR_stations[p[0]][0].split(':')[0].split('.')[2]
            IR_lat = float(IR_stations[p[0]][0].split(':')[1])
            IR_lon = float(IR_stations[p[0]][1])

    #     print('IR station ' + IR_sta + ': ' + str(IR_lat) + ', ' + str(IR_lon))

        try: # see if the IR station is in the modern_stations list
            w = np.where(modern_stations[:,1] == IR_sta)[0][0]
    #         print(w)
    #         print('IR sta: ' + IR_sta)
    #         print('Modern sta: ' + str(modern_stations[w,1]))
            status = True
            success = False

        except:
            print('No IR station ' + IR_sta + ' in modern BHZ list')
            print('Trying to access data anyway...')
            net = '*'
            sta = IR_sta
            loc = '*'
            a = 0
            b = 0

            try:
                E = client.get_waveforms(network = net, station = sta, location = loc, channel = 'BHE', starttime = arrtime - pre_post_sec, endtime = arrtime + pre_post_sec)
                net = E[0].stats.network
                hist_nets.append(net)
                hist_stas.append(sta)
                hist_lats.append(IR_lat)
                hist_lons.append(IR_lon)
                # E.plot()
#                 sta_savedir = event_savedir + net + '.' + sta + '.--/'
#                 if os.path.isdir(sta_savedir):
#                     pass
#                 else: # deletes directory to start over: shutil.rmtree(save_dir)  
#                     os.makedirs(sta_savedir)
#                 E.write(sta_savedir + 'BHE.mseed', format='MSEED')
                print('Saved BHE')
                a += 1
            except:
                print('No BHE')
                b += 1

            try:
                N = client.get_waveforms(network = net, station = sta, location = loc, channel = 'BHN', starttime = arrtime - pre_post_sec, endtime = arrtime + pre_post_sec)
                net = N[0].stats.network
                hist_nets.append(net)
                hist_stas.append(sta)
                hist_lats.append(IR_lat)
                hist_lons.append(IR_lon)
                # N.plot()
#                 sta_savedir = event_savedir + net + '.' + sta + '.--/'
#                 if os.path.isdir(sta_savedir):
#                     pass
#                 else: # deletes directory to start over: shutil.rmtree(save_dir)  
#                     os.makedirs(sta_savedir)
#                 N.write(sta_savedir + 'BHN.mseed', format='MSEED')
                print('Saved BHN')
                a += 1
            except:
                print('No BHN')
                b += 1

            try:
                Z = client.get_waveforms(network = net, station = sta, location = loc, channel = 'BHZ', starttime = arrtime - pre_post_sec, endtime = arrtime + pre_post_sec)
                net = Z[0].stats.network
                hist_nets.append(net)
                hist_stas.append(sta)
                hist_lats.append(IR_lat)
                hist_lons.append(IR_lon)
                # Z.plot()
#                 sta_savedir = event_savedir + net + '.' + sta + '.--/'
#                 if os.path.isdir(sta_savedir):
#                     pass
#                 else: # deletes directory to start over: shutil.rmtree(save_dir)  
#                     os.makedirs(sta_savedir)
#                 Z.write(sta_savedir + 'BHZ.mseed', format='MSEED')
                print('Saved BHZ')
                a += 1
            except:
                print('No BHZ')
                b += 1

            if b == 3:
                status = False

            if a > 0:
                success = True

        if status == False:
            print('Skipping - no data available')
            continue # skips the rest of this loop if there's no data available

        if success == True:
            continue # skips the rest of the loop if the data was already downloaded

        modern_net = modern_stations[w,0]
        modern_sta = modern_stations[w,1]
        modern_loc = modern_stations[w,2]
        modern_stacode = modern_net + '.' + modern_sta + '.' + modern_loc

        modern_lat = float(modern_stations[w,4])

        if modern_stations[w][1] in bad_format:
            modern_lon = float(modern_stations[w,5][0:9])
        else:
            modern_lon = float(modern_stations[w,5])

    #     print('Modern station ' + str(modern_stacode) + ': ' + str(modern_lat) + ', ' + str(modern_lon))

        if len(p) == 0: # from above - there will be no IR_lat or IR_lon
            j = False
            k = False

        else:
            j = np.isclose(IR_lat, modern_lat, atol = 0.0005)
            k = np.isclose(IR_lon, modern_lon, atol = 0.0005)

        if j == True and k == True: # Station name and location matches
    #         print('IR station ' + IR_sta + ': (' + str(IR_lat) + ', ' + str(IR_lon) + ')')
    #         print('Modern station ' + modern_stacode + ': (' + str(modern_lat) + ', ' + str(modern_lon) + ')')
            net = modern_net
            sta = modern_sta
            loc = modern_loc

        elif j == False and k == False: # Station name matches, but not location (like if it was missing from the IR file)
    #         print('IR station ' + IR_sta + '; not in IR file')
    #         print('Modern station ' + modern_stacode + ': ' + str(modern_lat) + ', ' + str(modern_lon))
            net = modern_net
            sta = modern_sta
            loc = modern_loc

        else:
            print('Locations do not match')
            status = False

        if status == False:
            print('Skipping')
            continue # skips the rest of this loop if the station locations don't match

        print(net + '.' + sta + '.' + loc)

        print('Getting data...')

        try:
            E = client.get_waveforms(network = net, station = sta, location = loc, channel = 'BHE', starttime = arrtime - pre_post_sec, endtime = arrtime + pre_post_sec)
            # E.plot()
            hist_nets.append(net)
            hist_stas.append(sta)
            hist_lats.append(modern_lat)
            hist_lons.append(modern_lon)
#             sta_savedir = event_savedir + net + '.' + sta + '.' + loc + '/'
#             if os.path.isdir(sta_savedir):
#                 pass
#             else: # deletes directory to start over: shutil.rmtree(save_dir)  
#                 os.makedirs(sta_savedir)
#             E.write(sta_savedir + 'BHE.mseed', format='MSEED')
            print('Saved BHE')
        except:
            print('No BHE')

        try:
            N = client.get_waveforms(network = net, station = sta, location = loc, channel = 'BHN', starttime = arrtime - pre_post_sec, endtime = arrtime + pre_post_sec)
            # N.plot()
            hist_nets.append(net)
            hist_stas.append(sta)
            hist_lats.append(modern_lat)
            hist_lons.append(modern_lon)
#             sta_savedir = event_savedir + net + '.' + sta + '.' + loc + '/'
#             if os.path.isdir(sta_savedir):
#                 pass
#             else: # deletes directory to start over: shutil.rmtree(save_dir)  
#                 os.makedirs(sta_savedir)
#             N.write(sta_savedir + 'BHN.mseed', format='MSEED')
            print('Saved BHN')
        except:
            print('No BHN')

        try:
            Z = client.get_waveforms(network = net, station = sta, location = loc, channel = 'BHZ', starttime = arrtime - pre_post_sec, endtime = arrtime + pre_post_sec)
            # Z.plot()
            hist_nets.append(net)
            hist_stas.append(sta)
            hist_lats.append(modern_lat)
            hist_lons.append(modern_lon)
#             sta_savedir = event_savedir + net + '.' + sta + '.' + loc + '/'
#             if os.path.isdir(sta_savedir):
#                 pass
#             else: # deletes directory to start over: shutil.rmtree(save_dir)  
#                 os.makedirs(sta_savedir)
#             Z.write(sta_savedir + 'BHZ.mseed', format='MSEED')
            print('Saved BHZ')
        except:
            print('No BHZ')

#     with open(event_savedir + 'mult_locs.txt', 'w') as f:
#         for line in mult_locs:
#             f.write(f"{line}\n")

In [ ]:
print(hist_nets)
print(hist_stas)
print(hist_lats)
print(hist_lons)

print(len(hist_nets))
print(len(hist_stas))
print(len(hist_lats))
print(len(hist_lons))